# Data Science Notebook Session에 HDFS Connector 설정

## 1. 사전 준비

### 1.1 OCI CLI 인증 설정 

- Notebook Session에 OCI CLI 인증 설정을 선행해야 함

In [ ]:
!oci os ns get

- 위 cell의 실행 결과가 다음과 같은 유형이 아닐 경우, OCI CLI 설정에 문제가 있습니다. 
- 아래 코드를 실행하여 OCI CLI 설정을 완성하시기 바랍니다.

```
{
  "data": "cn3cwx1nbt48"
}
```

- OCI CLI 설정이 안된 경우 다음 코드를 Notebook Cell에서 실행

```
%%bash 
bash -c "$(curl -L http://taewan.kim/shells/ods-ocicli-setup.sh)"
```

### 1.2 PySpark Conda 환경 구성 

- HDFS Connector 설정에 앞서 PySpark Conda 환경 구성이 선행되어야 함
- PySpark Conda 환경이 구성되지 않았다면 
    - __/data-science-quickstart/02-ods-pyspark-setup.ipynb__ 노트북을 참조하여 설치

## 2. HDFS Connector 환결 설정

- Notebook Cell에 다음 명령을 입력하여 HDFS Connector 환경 설정을 수행합니다. 

```
%%bash 
bash -c "$(curl -L http://taewan.kim/shells/ods-hdfs-connector-setup.sh)"
```

In [ ]:
%%bash 
bash -c "$(curl -L http://taewan.kim/shells/ods-hdfs-connector-setup.sh)"

## 3. PySpark 테스트 환경 구성

### 3.1 테스트 수행 컴파트먼트 OCID 확인

- PySpark의 테스트 용 데이터를 Object Stoage에 올리기 위해서는 bucket을 먼저 만들어야 합니다. 
- Bucket을 만들기 위해서는 Compartment OCID가 필요합니다. 
- 다음 절차를 통해서 Compartment OCID를 확인합니다. 

![](https://taewanmerepo.github.io/2021/03/odsocicli/image0210.jpg)
![](https://taewanmerepo.github.io/2021/03/odsocicli/image0220.jpg)
![](https://taewanmerepo.github.io/2021/03/odsocicli/image0230.jpg)

### 3.2 demo_data Bucket 생성

- 위에서 확인한 Compartment ocide로 다음 Cell을 업데이트하고 수정합니다. 

```
%%bash

export COMPARTMENT_OCID=ocid1.compartment.oc1..aaaaaaaadyyakdngvhdtazx7k66m3kb66p6fphxy5nt26rxae5maoiqnevsq
oci os bucket create --name demo_data --compartment-id $COMPARTMENT_OCID
```

In [ ]:
%%bash

export COMPARTMENT_OCID=<Replace with your compartment ocid>
oci os bucket create --name demo_data --compartment-id $COMPARTMENT_OCID

### 3.3 demo_data Bucket에 데이터 로딩

In [ ]:
%%bash
export DATA_FILE=/opt/notebooks/ads-examples/oracle_data/oracle_traffic_timeseries_dataset1.csv 
oci os object put --bucket-name demo_data --file $DATA_FILE

- Data 파일 업로그 결과

![](https://taewanmerepo.github.io/2021/03/odsocicli/image0240.jpg)

## 4. PySpark 코드 실행

- 다음 코드는 Object Storage에서 데이터를 읽어오는 PySpark 예제 코드입니다.

- Object Storage Namespace를 확인하고 코드의 oci url을 완성합니다. 

```
!oci os ns get

{
  "data": "cn3cwx1nbt48"
}
```

```
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
file = sc.textFile("oci://demo_data@cn3cwx1nbt48/oracle_traffic_timeseries_dataset1.csv")
file.count()
```

In [ ]:
!oci os ns get

- 아래 Cell의 <Replce with your name space>를 위 Cell에서 확인한 Name Space로 업데이트하고 수행

In [ ]:

from pyspark import SparkContext
sc = SparkContext.getOrCreate()
file = sc.textFile("oci://demo_data@<Replce with your name space>/oracle_traffic_timeseries_dataset1.csv")
file.take(10)

In [ ]:
file.take(10)

- 위 코드 실행결과

```
['rain_amt_last_hr,holiday,sensor9,sensor4,temperature_k,weather,snow_amt_last_hr,sensor3,sensor7,sensor0,date,sensor1,cloud_coverage,sensor2,sensor6,sensor8,sensor5,405_CAR_COUNT',
 '0.0,None,11.32,2.13,290.72,LIGHT_RAIN,0.0,0.60,15.88,21.22,1988-10-31 17:00:00,0.47,92,1.94,0.0,4.07,2.78,26275',
 '0.0,None,10.15,2.42,298.54,BROKEN_CLOUDS,0.0,2.28,6.91,29.57,1985-07-07 05:00:00,1.18,56,1.36,1.0,3.26,2.95,2660',
 '0.0,None,7.93,2.27,302.85,SKY_IS_CLEAR,0.0,2.34,13.02,3.44,1986-06-06 19:00:00,0.96,0,2.86,1.0,3.22,3.02,20820',
 '0.0,None,15.77,2.12,290.88,BROKEN_CLOUDS,0.0,0.44,14.12,28.95,1987-08-19 19:00:00,2.42,75,2.72,0.0,4.35,3.02,16740',
 '0.0,None,13.94,2.59,293.21,SCATTERED_CLOUDS,0.0,0.06,6.97,6.63,1989-08-10 21:00:00,2.00,40,1.16,1.0,-0.11,2.95,16580',
 '0.0,None,14.42,2.14,289.55,BROKEN_CLOUDS,0.0,0.37,8.07,1.63,1987-10-04 13:00:00,0.91,75,3.34,1.0,1.78,2.98,30460',
 '0.0,None,15.11,2.04,281.86,OVERCAST_CLOUDS,0.0,1.77,7.89,41.50,1984-11-04 17:00:00,0.26,90,2.45,1.0,1.89,3.01,18645',
 '0.0,None,2.56,2.08,294.30,SCATTERED_CLOUDS,0.0,1.55,19.49,14.78,1990-06-10 07:00:00,1.49,40,1.14,0.0,3.66,3.09,5240',
 '0.0,None,8.75,2.56,292.22,SKY_IS_CLEAR,0.0,1.14,12.58,21.16,1985-05-24 20:00:00,2.40,1,3.11,1.0,2.81,2.87,14325']
```